In [1]:
import pandas as pd
import bs4
import requests
import json
import re
import tqdm

import scipy.stats as st
import numpy as np

import bokeh
from bokeh.plotting import show as show_interactive
from bokeh.plotting import output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import CustomJS, TextInput, LassoSelectTool, Select, MultiSelect, ColorBar, Legend, LegendItem
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Button, HTMLTemplateFormatter
from bokeh.events import SelectionGeometry
from bokeh.transform import linear_cmap, jitter
from bokeh.models import FactorRange


In [2]:
# As of 2020 https://www.ncbi.nlm.nih.gov/research/cog/
COG_dict = {
    "A" : "RNA processing and modification",
    "B" : "Chromatin structure and dynamics",
    "C" : "Energy production and conversion",
    "D" : "Cell cycle control, cell division, chromosome partitioning",
    "E" : "Amino acid transport and metabolism",
    "F" : "Nucleotide transport and metabolism",
    "G" : "Carbohydrate transport and metabolism",
    "H" : "Coenzyme transport and metabolism",
    "I" : "Lipid transport and metabolism",
    "J" : "Translation, ribosomal structure and biogenesis",
    "K" : "Transcription",
    "L" : "Replication, recombination, and repair",
    "M" : "Cell wall/membrane/envelope biogenesis",
    "N" : "Cell motility",
    "O" : "Posttranslational modification, protein turnover, chaperones",
    "P" : "Inorganic ion transport and metabolism",
    "Q" : "Secondary metabolites biosynthesis, transport and catabolism",
    "T" : "Signal transduction mechanisms",
    "U" : "Intracellular trafficking, secretion, and vesicular transport",
    "V" : "Defense mechanisms",
    "W" : "Extracellular structures",
    "X" : "Mobilome: prophages, transposons",
    "Y" : "Nuclear structure",
    "Z" : "Cytoskeleton",
    "R" : "General function prediction only",
    "S" : "Function unknown",
}

In [3]:
def term_count_dict_from_annotation_df(annot_df, term_column):
    
    column = annot_df[term_column].values
    
    funct_terms = []
    for entry in column:
        if entry != '-':
            if term_column == 'COG_category':
                # terms = [f'{e}: {COG_dict[e]}' for e in entry]
                terms = list(entry)
            else:
                terms = entry.split(',')
            for t in terms:
                funct_terms.append(t)

#     len(terms)
    
    term_count_dict = {}
    
    for t in funct_terms:
        count = term_count_dict.get(t, 0)
        count += 1
        term_count_dict[t] = count
        
    return term_count_dict

In [4]:
def enrichment_analysis(module, leiden_label_df, phases, background_annotation, term_column):
    
    module_ttids = leiden_label_df.loc[leiden_label_df[f'leiden_label_{phases}'] == module]['TTHERM_ID'].values
    
    module_annotation = background_annotation.loc[background_annotation['TTHERM_ID'].isin(module_ttids)]
    
    background_term_dict = term_count_dict_from_annotation_df(background_annotation, term_column)
    module_term_dict = term_count_dict_from_annotation_df(module_annotation, term_column)
    
    bs = []
    ps = []
    folds = []
    terms = []
    
    for t, module_count in module_term_dict.items():
        
        background_count = background_term_dict[t]
        module_size = len(module_annotation)
        background_size = len(background_annotation)
        
        standard_contingency_table = [
                                [module_count, background_count - module_count], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        # The -1 and +1 make this more conservative (see explanation from the DAVID database: 
        # https://david.ncifcrf.gov/helps/functional_annotation.html#geneenrich)
        conservative_contingency_table = [
                                [module_count - 1, background_count - module_count + 1], 
                                [module_size - module_count, background_size - module_size - (background_count - module_count)]
                            ]
        
        
        odds, p_standard = st.fisher_exact(standard_contingency_table, 'greater')
        odds, p_conservative = st.fisher_exact(conservative_contingency_table, 'greater')
        
        p_reasonable = np.mean([p_standard, p_conservative])
        
        bonferroni  = p_reasonable * len(module_term_dict)

        fold_enrichment = (module_count/module_size) / (background_count/background_size)

        if bonferroni <= 0.05:
            
            ps.append(p_reasonable)
            bs.append(bonferroni)
            folds.append(fold_enrichment)
            terms.append(t)
            
#         else:
#             ps.append('')
#             bs.append('')
#             folds.append('')
#             terms.append('')
            
    return ps, bs, folds, terms
            
            

In [5]:
def get_GO_info(go_term):
    
    r = requests.get(f'https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_term}/complete', 'html5lib')
    
    go_info = json.loads(r.text)
    
    name = go_info['results'][0]['name']
    
    definition = go_info['results'][0]['definition']['text']
    
    obsolete = go_info['results'][0]['isObsolete']
    
    return name, definition, obsolete

In [6]:
def get_KEGG_info(term):
    try:
#         dfs = pd.read_html(f'https://www.genome.jp/entry/{term}')
        response = requests.get(f'https://rest.kegg.jp/get/{term}')
    except:
        return "NA"

    text = response.text
    try:
        name_string = re.search(r'NAME\s*[\w \-]*', response.text).group()
        name = re.split(r'\s\s+', name_string)[1]
    except:
        name = 'poop'
#     data = dfs[3]
#     entry = data.loc[data[0] == 'Entry'][1].values[0]
#     name = data.loc[data[0] == 'Name'][1].values[0]

    if name == 'poop':
        print(term)
    
    return name

In [7]:
def get_EC_info(term):
    
    r = requests.get(f"https://www.ebi.ac.uk/enzymeportal/rest/enzymes/{term}")
    try:
        name = r.json()['enzymeName']
    except:
        name = 'NA'
    
    return name

In [8]:
def write_enrichment(lldf, phases, out_prefix, term_column):
    
    with open(f'../enrichment/{out_prefix}_enrichment.txt', 'w') as fl:
    
        for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):

            # print(f'Module {m}')
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, complete_annot, term_column)

            fl.write(f'Module {m}\n')
            for b, fold, t in zip(bs, folds, terms):
                
                if term_column == 'COG_category':
                    
                    definition = COG_dict[t]
                    t = f'{t}: {definition}'
                
                elif term_column == 'GOs':
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
                    else:
                        t = f'{name.capitalize()} ({t}): {definition}'
                        
                elif term_column.split('_')[0] == 'KEGG':
                    name = get_KEGG_info(t)
                    
                    t = f'{t}: {name}'
                    
                elif term_column == 'EC':
                    
                    name = get_EC_info(t)
                    t = f'{t}: {name}'
                    
                        
                # print(f"{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n")

                fl.write(f'{t}\nFold-enrichment: {fold:.02f}\nBonferroni-corrected p-value: {b:.02e}\n\n')

In [9]:


response = requests.get('https://rest.kegg.jp/get/ko:K19607')

In [10]:
test = response.text.split('\n')

In [11]:
name = re.search(r'NAME\s*[\w \-]*', response.text).group()

In [12]:
re.split(r'\s\s+', name)

['NAME', 'X-linked retinitis pigmentosa GTPase regulator']

In [13]:
name

'NAME        X-linked retinitis pigmentosa GTPase regulator'

In [14]:
test[:10]

['ENTRY       K19607                      KO',
 'SYMBOL      RPGR',
 'NAME        X-linked retinitis pigmentosa GTPase regulator',
 'DISEASE     H00481  Cone-rod dystrophy and cone dystrophy',
 '            H00527  Retinitis pigmentosa',
 'BRITE       KEGG Orthology (KO) [BR:ko00001]',
 '             09180 Brite Hierarchies',
 '              09183 Protein families: signaling and cellular processes',
 '               03037 Cilium and associated proteins',
 '                K19607  RPGR; X-linked retinitis pigmentosa GTPase regulator']

In [15]:
r = requests.get("https://www.ebi.ac.uk/enzymeportal/rest/enzymes/2.7.11.1")
r.json()['enzymeName']

'Non-specific serine/threonine protein kinase'

In [16]:
def get_enrichment_df(lldf, phases, background_annotation, term_columns=['COG_category', 'GOs', 'KEGG_ko', 'EC'], outfile=None):
    
    module_dfs = []
    
    for m in tqdm.tqdm(sorted(lldf[f'leiden_label_{phases}'].unique())):
        
        term_dfs = []

        for tc in term_columns:
        
            ps, bs, folds, terms = enrichment_analysis(m, lldf, phases, background_annotation, tc)

            # fl.write(f'Module {m}\n')
            
            info = []

            if tc == 'GOs':

                for t in terms:
                    name, definition, obsolete = get_GO_info(t)
                    if obsolete:
                        info.append(f'{name.capitalize()}: {definition} (obsolete)')
                    else:
                        info.append(f'{name.capitalize()}: {definition}')
                        
            elif tc == 'COG_category':
                for t in terms:
                    info.append(COG_dict[t])
                                
            elif tc == 'KEGG_ko':
                for t in terms:
                    info.append(get_KEGG_info(t))
                    
            elif tc == 'EC':
                for t in terms:
                    info.append(get_EC_info(t))
                    
            term_df = pd.DataFrame({'module': [m]*len(terms),
                                    'term': terms,
                                    'info': info,
                                    'fold_change': folds,
                                    'bonferroni': bs})
            
            term_dfs.append(term_df)
            
        module_df = pd.concat(term_dfs)
        
        module_dfs.append(module_df)
        
    all_enrichment_df = pd.concat(module_dfs)
    
    if outfile:
        all_enrichment_df.to_csv(outfile, index=False)
    
    return all_enrichment_df
        
        
                        
            
                    
            
        
#         for b, fold, t in zip(bs, folds, terms):

#             if term_column == 'GOs':
#                 name, definition, obsolete = get_GO_info(t)
#                 if obsolete:
#                     t = f'{name.capitalize()} ({t}): {definition} (obsolete)'
#                 else:
#                     t = f'{name.capitalize()} ({t}): {definition}'

#             if term_column.split('_')[0] == 'KEGG':
#                 dfs = pd.read_html(f'https://www.genome.jp/entry/{t}')

#                 data = dfs[4]
#                 entry = data.loc[data[0] == 'Entry'][1].values[0]
#                 name = data.loc[data[0] == 'Name'][1].values[0]

#                 t = f'{t}: {name}'

In [17]:
complete_annotation = pd.read_csv('../eggnog/complete_eggnog_annotation.csv')

In [18]:
lldf_nn3 = pd.read_csv('../embedding/mcl_rcl_label_df_round_1.csv')

In [28]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_full'].values[0])

72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72


In [19]:
lldf_nn3['leiden_label_full'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [20]:
%pdb

Automatic pdb calling has been turned ON


In [21]:
enrich_full_nn3 = get_enrichment_df(lldf_nn3, 'full', complete_annotation, outfile='./test_nn3_full_enrichment.csv')

  0%|          | 0/809 [00:00<?, ?it/s]

 92%|█████████▏| 742/809 [05:16<00:11,  6.06it/s]  

ko:K10827


100%|██████████| 809/809 [05:28<00:00,  2.47it/s]


In [22]:
enrich_full_nn3 = pd.read_csv('./test_nn3_full_enrichment.csv')

In [23]:
enrich_full_nn3.head()

,module,term,info,fold_change,bonferroni
0,0.0,B,Chromatin structure and dynamics,2.592711,0.018331
1,1.0,T,Signal transduction mechanisms,1.497861,0.020480
2,1.0,ko:K01373,cathepsin F,4.613411,0.006447
3,1.0,3.4.22.41,Cathepsin F,4.613411,0.001847
4,2.0,T,Signal transduction mechanisms,1.625530,0.007472


In [30]:
enrich_full_nn3.loc[enrich_full_nn3['module']==72]

,module,term,info,fold_change,bonferroni
127,72.0,ko:K01379,cathepsin D,65.645000,0.007851
128,72.0,ko:K01378,chymosin,437.633333,0.037877
129,72.0,3.4.23.5,Cathepsin D,65.645000,0.003695
130,72.0,3.5.4.4,Adenosine deaminase,187.557143,0.041596
131,72.0,3.4.23.1,Pepsin A,328.225000,0.023767
132,72.0,3.4.23.4,Chymosin,437.633333,0.017824


In [24]:
enrich_full_nn3['module'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,  10.,  11.,
        12.,  13.,  15.,  21.,  22.,  23.,  24.,  28.,  34.,  35.,  37.,
        39.,  40.,  41.,  44.,  46.,  48.,  49.,  53.,  57.,  58.,  60.,
        65.,  67.,  72.,  75.,  76.,  78.,  82.,  85.,  88.,  89.,  91.,
        94.,  95.,  96.,  98.,  99., 102., 104., 108., 112., 122., 124.,
       129., 133., 134., 141., 146., 147., 148., 151., 153., 159., 160.,
       163., 164., 168., 170., 171., 175., 185., 187., 189., 192., 194.,
       199., 200., 204., 209., 211., 215., 228., 236., 242., 243., 257.,
       263., 278., 280., 281., 287., 290., 294., 299., 302., 304., 315.,
       320., 321., 323., 326., 330., 336., 345., 367., 370., 372., 377.,
       384., 385., 398., 402., 413., 431., 432., 443., 463., 467., 468.,
       474., 476., 481., 485., 488., 490., 507., 513., 528., 529., 557.,
       563., 568., 577., 588., 589., 595., 596., 599., 602., 610., 623.,
       630., 634., 642., 643., 645., 662., 684., 69

In [25]:
lldf_nn3['leiden_label_full'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [26]:
lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == 'TTHERM_01386050']

,Unnamed: 0,TTHERM_ID,leiden_label_full
12635,12635,TTHERM_01386050,170


In [29]:
gene_list_test = ['TTHERM_000013409', 'TTHERM_01321550', 'TTHERM_00011710', 'TTHERM_00321680', 'TTHERM_00355700', 'TTHERM_00938950', 'TTHERM_01372820', 'TTHERM_00013410', 'TTHERM_00390080', 'TTHERM_00516380', 'TTHERM_00038880', 'TTHERM_00059370', 'TTHERM_00473020', 'TTHERM_00497590', 'TTHERM_00558350', 'TTHERM_00052190', 'TTHERM_00392790', 'TTHERM_00410180', 'TTHERM_00685980', 'TTHERM_00445920', 'TTHERM_00471040', 'TTHERM_00140780', 'TTHERM_00145480', 'TTHERM_00321720', 'TTHERM_00628650', 'TTHERM_00526730', 'TTHERM_01156770', 'TTHERM_00312200', 'TTHERM_01332070', 'TTHERM_00318900', 'TTHERM_00340180', 'TTHERM_00592740', 'TTHERM_00440600', 'TTHERM_01321570', 'TTHERM_00537380', 'TTHERM_00585170', 'TTHERM_01197130', 'TTHERM_00554390', 'TTHERM_00649180', 'TTHERM_00691410', ]
for gene in gene_list_test:
    print((lldf_nn3.loc[lldf_nn3['TTHERM_ID'] == gene])['leiden_label_full'].values[0])

72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72
72


end here. Return to embedding notebook!